In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pyarrow.parquet as pq
import gc
import pywt
from statsmodels.robust import mad
import scipy
from scipy import signal
from scipy.signal import butter
from tqdm import tqdm_notebook
import os
from multiprocessing import Pool

from numba import jit

import warnings

# Suppress pandas future warnings, I am using different library versions locally
# that do not raise warnings.
warnings.simplefilter(action='ignore', category=FutureWarning)

data_dir = '../input'

In [15]:
print(scipy.__version__)

1.1.0


In [16]:
num_cpu = os.cpu_count()
pool = Pool(num_cpu)

Process ForkPoolWorker-9:
AttributeError: Can't get attribute 'main' on <module '__main__'>
Traceback (most recent call last):
  File "/home/cwademo123/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/cwademo123/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/cwademo123/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/cwademo123/anaconda3/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
Process ForkPoolWorker-10:
Traceback (most recent call last):
  File "/home/cwademo123/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/cwademo123/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/cwademo123/anaconda3/lib/python3.6/mult

In [17]:
# 800,000 data points taken over 20 ms
# Grid operates at 50hz, 0.02 * 50 = 1, so 800k samples in 20 milliseconds will capture one complete cycle
n_samples = 800000

# Sample duration is 20 miliseconds
sample_duration = 0.02

# Sample rate is the number of samples in one second
# Sample rate will be 40mhz
sample_rate = n_samples * (1 / sample_duration)

In [18]:
def maddest(d, axis=None):
    """
    Mean Absolute Deviation
    """
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

In [19]:
def high_pass_filter(x, low_cutoff=1000, sample_rate=sample_rate):
    """
    From @randxie https://github.com/randxie/Kaggle-VSB-Baseline/blob/master/src/utils/util_signal.py
    Modified to work with scipy version 1.1.0 which does not have the fs parameter
    """
    
    # nyquist frequency is half the sample rate https://en.wikipedia.org/wiki/Nyquist_frequency
    nyquist = 0.5 * sample_rate
    norm_low_cutoff = low_cutoff / nyquist
    
    # Fault pattern usually exists in high frequency band. According to literature, the pattern is visible above 10^4 Hz.
    # scipy version 1.2.0
    #sos = butter(10, low_freq, btype='hp', fs=sample_fs, output='sos')
    
    # scipy version 1.1.0
    sos = butter(10, Wn=[norm_low_cutoff], btype='highpass', output='sos')
    filtered_sig = signal.sosfilt(sos, x)

    return filtered_sig

In [20]:
def denoise_signal( x, wavelet='db4', level=1):
    """
    1. Adapted from waveletSmooth function found here:
    http://connor-johnson.com/2016/01/24/using-pywavelets-to-remove-high-frequency-noise/
    2. Threshold equation and using hard mode in threshold as mentioned
    in section '3.2 denoising based on optimized singular values' from paper by Tomas Vantuch:
    http://dspace.vsb.cz/bitstream/handle/10084/133114/VAN431_FEI_P1807_1801V001_2018.pdf
    """
    
    # Decompose to get the wavelet coefficients
    coeff = pywt.wavedec( x, wavelet, mode="per", level=level)
    
    # Calculate sigma for threshold as defined in http://dspace.vsb.cz/bitstream/handle/10084/133114/VAN431_FEI_P1807_1801V001_2018.pdf
    # As noted by @harshit92 MAD referred to in the paper is Mean Absolute Deviation not Median Absolute Deviation
    sigma = (1/0.6745) * maddest( coeff[-level] )

    # Calculte the univeral threshold
    uthresh = sigma * np.sqrt( 2*np.log( len( x ) ) )
    coeff[1:] = ( pywt.threshold( i, value=uthresh, mode='hard' ) for i in coeff[1:] )
    
    # Reconstruct the signal using the thresholded coefficients
    return pywt.waverec( coeff[1:], wavelet, mode='per' )

In [21]:
def denoise_signal_2( x, wavelet='db4', level=1):
    """
    1. Adapted from waveletSmooth function found here:
    http://connor-johnson.com/2016/01/24/using-pywavelets-to-remove-high-frequency-noise/
    2. Threshold equation and using hard mode in threshold as mentioned
    in section '3.2 denoising based on optimized singular values' from paper by Tomas Vantuch:
    http://dspace.vsb.cz/bitstream/handle/10084/133114/VAN431_FEI_P1807_1801V001_2018.pdf
    """
    
    # Decompose to get the wavelet coefficients
    coeff = pywt.wavedec( x, wavelet, mode="per", level=level)
    
    # Calculate sigma for threshold as defined in http://dspace.vsb.cz/bitstream/handle/10084/133114/VAN431_FEI_P1807_1801V001_2018.pdf
    # As noted by @harshit92 MAD referred to in the paper is Mean Absolute Deviation not Median Absolute Deviation
    sigma = (1/0.6745) * maddest( coeff[-level] )

    # Calculte the univeral threshold
    uthresh = sigma * np.sqrt( 2*np.log( len( x ) ) )
    coeff[1:] = ( pywt.threshold( i, value=uthresh, mode='hard' ) for i in coeff[1:] )
    
    # Reconstruct the signal using the thresholded coefficients
    return pywt.waverec( coeff[0:], wavelet, mode='per' )

In [22]:
def remove_corona(x_dn):
    index = pd.Series(x_dn).loc[np.abs(x_dn)>0].index
    corona_idx = []
    for idx in index:
        for i in range(1,maxDistance+1):
            if idx+i < pd.Series(x_dn).shape[0]:
                if x_dn[idx+i]/(x_dn[idx]+1e-04)<-maxHeightRatio:
                    x_dn[idx:idx+maxTicksRemoval] = 0
                    corona_idx.append(idx)
    return x_dn, corona_idx

In [23]:
def main(x_df):
    X_HP = []
    X_DN = []

    X_HP.append(high_pass_filter(x_df.iloc[:, 0], low_cutoff=10000, sample_rate=sample_rate))
    X_HP.append(high_pass_filter(x_df.iloc[:, 1], low_cutoff=10000, sample_rate=sample_rate))
    X_HP.append(high_pass_filter(x_df.iloc[:, 2], low_cutoff=10000, sample_rate=sample_rate))

    X_DN.append(denoise_signal(X_HP[0], wavelet='haar', level=1))
    X_DN.append(denoise_signal(X_HP[1], wavelet='haar', level=1))
    X_DN.append(denoise_signal(X_HP[2], wavelet='haar', level=1))

    X_DN[0], corona_idx = remove_corona(X_DN[0])
    X_DN[1], corona_idx = remove_corona(X_DN[1])
    X_DN[2], corona_idx = remove_corona(X_DN[2])

    total = X_DN[0] + X_DN[1] + X_DN[2]
    total = pd.Series(total)

    meanamppos_id[col:col+3] = np.mean(total[total>0])
    meanampneg_id[col:col+3] = np.mean(total[total<0])

    stdamp_id[col:col+3] = np.std(total)
    stdamppos_id[col:col+3] = np.std(total[total>0])
    stdampneg_id[col:col+3] = np.std(total[total<0])

In [24]:
maxDistance = 10
maxHeightRatio = 0.25
maxTicksRemoval =500

In [25]:
meanamppos_id_test = []
meanampneg_id_test = []

stdamp_id_test = []
stdamppos_id_test = []
stdampneg_id_test = []

In [33]:
for i in range(7):
    subset_test = pq.read_pandas('../input/test.parquet', columns=[str(i*3000 + j + 8712) for j in range(3000)]).to_pandas()
    
    meanamppos_id = np.zeros(subset_test.shape[1])
    meanampneg_id = np.zeros(subset_test.shape[1])
    stdamp_id = np.zeros(subset_test.shape[1])
    stdamppos_id = np.zeros(subset_test.shape[1])
    stdampneg_id = np.zeros(subset_test.shape[1])
    
    if __name__ == '__main__':
        results = pool.map(main, (subset_test.iloc[:, col:col+3] for col in tqdm_notebook(range(0, subset_test.shape[1], 3))))
    #main(subset_test)
    
    meanamppos_id_test = np.hstack((meanamppos_id_test, meanamppos_id))
    meanampneg_id_test = np.hstack((meanampneg_id_test, meanampneg_id))
    stdamp_id_test = np.hstack((stdamp_id_test, stdamp_id))
    stdamppos_id_test = np.hstack((stdamppos_id_test, stdamppos_id))
    stdampneg_id_test = np.hstack((stdampneg_id_test, stdampneg_id))
    
    gc.collect()

KeyboardInterrupt: 

In [32]:
meanamppos_id

array([0., 0., 0., ..., 0., 0., 0.])

In [13]:
np.save('../features/meanamppos_id_test.npy', meanamppos_id_test)
np.save('../features/meanampneg_id_test.npy', meanampneg_id_test)

np.save('../features/stdamp_id_test.npy', stdamp_id_test)
np.save('../features/stdamppos_id_test.npy', stdamppos_id_test)
np.save('../features/stdampneg_id_test.npy', stdampneg_id_test)

In [14]:
pd.Series(numpeaks_id_test)

0         936.0
1         936.0
2         936.0
3           0.0
4           0.0
5           0.0
6           4.0
7           4.0
8           4.0
9          78.0
10         78.0
11         78.0
12         19.0
13         19.0
14         19.0
15          4.0
16          4.0
17          4.0
18        168.0
19        168.0
20        168.0
21          0.0
22          0.0
23          0.0
24         50.0
25         50.0
26         50.0
27         40.0
28         40.0
29         40.0
          ...  
20307      40.0
20308      40.0
20309      40.0
20310     430.0
20311     430.0
20312     430.0
20313       0.0
20314       0.0
20315       0.0
20316     126.0
20317     126.0
20318     126.0
20319    1907.0
20320    1907.0
20321    1907.0
20322     430.0
20323     430.0
20324     430.0
20325      65.0
20326      65.0
20327      65.0
20328      45.0
20329      45.0
20330      45.0
20331     855.0
20332     855.0
20333     855.0
20334     165.0
20335     165.0
20336     165.0
Length: 20337, dtype: fl